# Introduction to basic CUDA code generation

This notebook demonstrates CUDA code generation for some basic neuron and synapse models.

In [1]:
import inspect
import sys

sys.path.append('../../neural')

from synapse import *
from neuron import *
from cuda import CudaGenerator

## Hodgkin-Huxley Neuron

In [2]:
hh = HodgkinHuxley()
print "%s" % inspect.getsource(hh.ode)

    def ode(self, **kwargs):
        stimulus = kwargs.pop('stimulus', 0)

        alpha = np.exp(-(self.v+55.)/10.)-1.
        beta = (0.125*np.exp(-(self.v+65.)/80.))
        if abs(alpha) <= 1e-7:
            self.d_n = 0.1 * (1.-self.n) - beta * self.n
        else:
            self.d_n = (-0.01*(self.v+55.)/alpha) * (1.-self.n) - beta * self.n

        alpha = np.exp(-(self.v+40.)/10.)-1.
        beta = (4.*np.exp(-(self.v+65.)/18.))
        if abs(alpha) <= 1e-7:
            self.d_m = (1.-self.m) - beta * self.m
        else:
            self.d_m = (-0.1*(self.v+40.)/alpha) * (1.-self.m) - beta * self.m

        alpha = (0.07 * np.exp(-(self.v+65.)/20.))
        beta = 1. / (np.exp(-(self.v+35.)/10.)+1.)
        self.d_h = alpha * (1-self.h) - beta * self.h

        i_na = self.gNa * np.power(self.m, 3) * self.h * (self.v - self.ENa)
        i_k = self.gK * np.power(self.n, 4) * (self.v - self.EK)
        i_l = self.gL * (self.v - self.EL)

        self.d_v = stimulus - i_na - i

#### CUDA Code

In [3]:
code_generator = CudaGenerator(hh)
code_generator.generate()
print code_generator.cuda_src


#define  EL		-54.387
#define  GNA		120.0
#define  EK		-77.0
#define  ENA		50.0
#define  GL		0.3
#define  GK		36.0

#define  H_MIN		0.0
#define  H_MAX		1.0
#define  N_MIN		0.0
#define  N_MAX		1.0
#define  M_MIN		0.0
#define  M_MAX		1.0
#define  V_MIN		-80
#define  V_MAX		30

struct States {
    float h;
    float n;
    float m;
    float v;
};

struct Inters {
    float spike;
};

__device__ void clip(States &states)
{
    states.h = fmaxf(states.h, H_MIN);
    states.h = fminf(states.h, H_MAX);
    states.n = fmaxf(states.n, N_MIN);
    states.n = fminf(states.n, N_MAX);
    states.m = fmaxf(states.m, M_MIN);
    states.m = fminf(states.m, M_MAX);
    states.v = fmaxf(states.v, V_MIN);
    states.v = fminf(states.v, V_MAX);
}

__device__ void forward(
    States &states,
    States &gstates,
    float dt
)
{
    states.h += dt * gstates.h;
    states.n += dt * gstates.n;
    states.m += dt * gstates.m;
    states.v += dt * gstates.v;
}

__device__ int ode(
    States &states,
    Sta

## Alpha Synapse

In [4]:
alpha = Alpha()
print "%s" % inspect.getsource(hh.ode)

    def ode(self, **kwargs):
        stimulus = kwargs.pop('stimulus', 0)

        alpha = np.exp(-(self.v+55.)/10.)-1.
        beta = (0.125*np.exp(-(self.v+65.)/80.))
        if abs(alpha) <= 1e-7:
            self.d_n = 0.1 * (1.-self.n) - beta * self.n
        else:
            self.d_n = (-0.01*(self.v+55.)/alpha) * (1.-self.n) - beta * self.n

        alpha = np.exp(-(self.v+40.)/10.)-1.
        beta = (4.*np.exp(-(self.v+65.)/18.))
        if abs(alpha) <= 1e-7:
            self.d_m = (1.-self.m) - beta * self.m
        else:
            self.d_m = (-0.1*(self.v+40.)/alpha) * (1.-self.m) - beta * self.m

        alpha = (0.07 * np.exp(-(self.v+65.)/20.))
        beta = 1. / (np.exp(-(self.v+35.)/10.)+1.)
        self.d_h = alpha * (1-self.h) - beta * self.h

        i_na = self.gNa * np.power(self.m, 3) * self.h * (self.v - self.ENa)
        i_k = self.gK * np.power(self.n, 4) * (self.v - self.EK)
        i_l = self.gL * (self.v - self.EL)

        self.d_v = stimulus - i_na - i

#### CUDA Code

In [5]:
code_generator = CudaGenerator(alpha)
code_generator.generate()
print code_generator.cuda_src


#define  AR		12.5
#define  AD		12.19
#define  GMAX		1.0


struct States {
    float s;
    float u;
};


__device__ void forward(
    States &states,
    States &gstates,
    float dt
)
{
    states.s += dt * gstates.s;
    states.u += dt * gstates.u;
}

__device__ int ode(
    States &states,
    States &gstates,
    float &stimulus
)
{
    float tmp;

    gstates.s = states.u;
    tmp = (AR * AD);
    gstates.u = (((-(AR + AD)) * states.u) - (tmp * states.s));
    if (stimulus) {
        states.u = (states.u + tmp);
    }
    return 0;
}



__global__ void Alpha (
    int num_thread,
    float dt,
    float *g_s,
    float *g_u,
    float *g_stimulus
)
{
    /* TODO: option for 1-D or 2-D */
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid > num_thread)
        return;

    States states, gstates;

    /* import data */
    states.s = g_s[tid];
    states.u = g_u[tid];
    float stimulus = g_stimulus[tid];

    
    /* compute gradient */
    ode(states, gstates, sti

## Oizumi Model

In [6]:
class Oizumi(Model):
    Default_Inters = {'p': 0., 'NT':0}
    Default_States = {'N': (51., 0., 51.), 'g': (0., 0., 1000.)}
    Default_Params = {'N0': 51, 'q': 1.07, 'tauO': 10, 'tauN0': 100, 'pmax': 0.79, 'Kpre': 0.0035}
    
    def ode(self, input=0, f=0.):
        """
        Arguments:
            input (bool): spike indicator.
            f (float): overall spike rate.
        """
        self.p = self.pmax * np.exp(-self.Kpre * f)
        self.d_N = (self.N0 - self.N) ** 2 / self.tauN0
        self.d_g = -self.g / self.tauO
        
        if input == 1:
            self.NT = self.N*self.p
            self.g += self.NT*self.q
            self.N -= self.NT
        else:
            self.NT = 0

    def get_conductance(self):
        return self.g
    
oz = Oizumi()
print "%s" % inspect.getsource(oz.ode)

    def ode(self, input=0, f=0.):
        """
        Arguments:
            input (bool): spike indicator.
            f (float): overall spike rate.
        """
        self.p = self.pmax * np.exp(-self.Kpre * f)
        self.d_N = (self.N0 - self.N) ** 2 / self.tauN0
        self.d_g = -self.g / self.tauO
        
        if input == 1:
            self.NT = self.N*self.p
            self.g += self.NT*self.q
            self.N -= self.NT
        else:
            self.NT = 0



#### CUDA Code

In [7]:
code_generator = CudaGenerator(oz)
code_generator.generate()
print code_generator.cuda_src


#define  Q		1.07
#define  KPRE		0.0035
#define  N0		51
#define  TAUO		10
#define  PMAX		0.79
#define  TAUN0		100

#define  G_MIN		0.0
#define  G_MAX		1000.0
#define  N_MIN		0.0
#define  N_MAX		51.0

struct States {
    float g;
    float N;
};

struct Inters {
    float p;
    float NT;
};

__device__ void clip(States &states)
{
    states.g = fmaxf(states.g, G_MIN);
    states.g = fminf(states.g, G_MAX);
    states.N = fmaxf(states.N, N_MIN);
    states.N = fminf(states.N, N_MAX);
}

__device__ void forward(
    States &states,
    States &gstates,
    float dt
)
{
    states.g += dt * gstates.g;
    states.N += dt * gstates.N;
}

__device__ int ode(
    States &states,
    States &gstates,
    Inters &inters,
    float &input,
    float &f
)
{

    inters.p = (PMAX * expf(((-KPRE) * f)));
    gstates.N = (powf((N0 - states.N), 2) / TAUN0);
    gstates.g = ((-states.g) / TAUO);
    if ((input == 1)) {
        inters.NT = (states.N * inters.p);
        states.g = (states.g + (inters.N